In [8]:
#selenium setting
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re

In [9]:
options = Options()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36')

In [10]:
driver = webdriver.Chrome(options=options)

In [11]:
driver.get('https://www.fmkorea.com/index.php?mid=stock&category=&search_keyword=&search_target=&sort_index=pop&order_type=desc&listStyle=list&page=1')

In [12]:
pattern = r"\s{2,}" 
#본문을 가져올 때, 의미없는 text를 제거하고 가져오기
#/s: /s는 공백 문자를 나타냄. 공백, 탭, 줄바꿈 등을 포함
#{2,} 앞의 문자나 그룹이 2번 이상 연속으로 반복될 때를 나타냄

In [ ]:
with open('fm_korea_Crawling.csv', 'w', encoding='utf-8', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['탭','제목', '링크', '인기글 날짜', '조회수', '추천', '본문'])

    for PAGE in range(1, 695):
        print(f'Page {PAGE} is being scraped...')
        
        URL = f'https://www.fmkorea.com/index.php?mid=stock&category=&search_keyword=&search_target=&sort_index=pop&order_type=desc&listStyle=list&page={PAGE}'
        driver.get(URL)

        rows = driver.find_element(By.CLASS_NAME, 'bd_lst').find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')[1:]
        #class로 찾기가 쉽지 않음. bdlst 찾은 후 tbody찾고 tr 끝가지 찾기. 첫번째 tr은 class에 관한 설명이라 제외

        if rows:
            for row in rows: #tr 리스트들 중에서 순서대로 각 행 tr 확인하는 작업
                try:
                    tab = row.find_element(By.CLASS_NAME, 'cate').text.strip() #탭 가져오기
                except:
                    tab = None
                try:
                    title = row.find_element(By.CLASS_NAME, 'title').find_element(By.TAG_NAME, 'a').text.strip()#title 텍스트
                except:
                    title = None
                try:
                    link = row.find_element(By.CLASS_NAME, 'title').find_element(By.TAG_NAME, 'a').get_attribute('href')#링크 가져오기
                except:
                    link = None
                try:
                    date = row.find_element(By.CLASS_NAME, 'time').text.strip() #날짜
                except:
                    date = None
                try:
                    views = row.find_element(By.CLASS_NAME, 'm_no').text.strip()#조회수
                except:
                    views = None
                try:
                    likes = row.find_element(By.CLASS_NAME, 'm_no_voted').text.strip() #추천수
                except:
                    likes = None

                new_tab_script = f"window.open('{link}');" #본문 들어가는 링크
                driver.execute_script(new_tab_script) #본문 실행
                time.sleep(3) 
                driver.switch_to.window(driver.window_handles[1]) #1본문 창 / 0은 헤드라인 창
                try:
                    content = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.TAG_NAME, 'article'))
                    )
                    content = content.text.strip() #article tag안에 있는 모든 text가져오기
                    content = re.sub(pattern, ' ', content) #정규표현식을 사용하여 ' '로 대체하기 content text에서
                    content = content.replace('\n', ' ') #개행문자도 공백으로 변경

                except:
                    content = None

                driver.close()
                driver.switch_to.window(driver.window_handles[0]) #원래 창으로 돌아가기

                csv_writer.writerow([tab, title, link, date, views, likes, content])#파일에 쓰기
                print(tab, title, link, date, views, likes, content)#print문을 통하여 출력이 잘 되고 있는지 확인
        


